## Goal: Data processing, feature engineering and region-wise data distribution/export

#### - import necessary libraires

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import glob
import zipfile

#### - Data import from local download bucket

In [ ]:
# create list of the zipfiles
ziplist = glob.glob('./bucket/*.zip')
ziplist = [fil.split('/')[-1] for fil in ziplist]
ziplist

In [ ]:
# extract csv files from all zip files
path = './bucket/'
for x in ziplist:
    path = './bucket/' + x
    handle = zipfile.ZipFile(path)
    handle.extractall('./csv/' + path[-4])

In [4]:
# specifying the path to csv files
csv_path = './csv/'
# csv files in the path
files = glob.glob(csv_path + "/*.csv")

In [ ]:
# defining an empty list to store 
# content
df_all = pd.DataFrame()
content = []

# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

In [7]:
# converting content to data frame
df_all = pd.concat(content)

In [ ]:
df_all.info()

In [ ]:
#explore first few rows
df_all.head(20)

In [ ]:
#explore last few rows
df_all.tail(20)

In [ ]:
#check from which year data schema has changed
df_all[df_all[''Start date''].isnan()]

In [9]:
# It seems, column names are different from the year 2021. let's separate all column with different names
df_2021 = df_all[['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id',
       'member_casual']]

In [10]:
# Remove empty rows and rename column name
df_2021 = df_2021.dropna(axis=0)

In [44]:
df_2021.shape

(4248529, 9)

In [11]:
df_2021.rename(columns={'started_at': 'Start date', 
                        'ended_at': 'End date', 
                        'start_station_name':'Start station', 
                        'start_station_id':'Start station number', 
                        'end_station_name':'End station', 
                        'end_station_id':'End station number', 
                        'member_casual':'Member type', 
                        'ride_id':'Bike number'
                       }, inplace=True)

In [46]:
df_2021.columns

Index(['Bike number', 'rideable_type', 'Start date', 'End date',
       'Start station', 'Start station number', 'End station',
       'End station number', 'Member type'],
      dtype='object')

In [12]:
# Keep required columns only
df_2021 = df_2021[['Bike number', 'Start date', 'End date',
       'Start station', 'Start station number', 'End station number',
       'End station']]

In [13]:
# select required columns only from the data upto 2020
df_mod = df_all[['Bike number', 'Start date', 'End date',
       'Start station', 'Start station number', 'End station number',
       'End station']]

In [14]:
# combine all 
df_mod2 = df_mod.append(df_2021)

/var/folders/5k/0rwnsb5j12zfcxbb1cv47y7h0000gn/T/ipykernel_97961/2811041073.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mod2 = df_mod.append(df_2021)


In [15]:
#df_full = df_mod2.dropna(axis=0)

In [16]:
df_full.tail(10)

,Bike number,Start date,End date,Start station,Start station number,End station number,End station
228337,D5302584D7352CDE,2020-07-14 10:13:39,2020-07-14 10:19:58,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228338,D620AFED5E667611,2020-07-14 12:05:36,2020-07-14 12:10:52,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228339,BAE31EADFDE50510,2020-07-17 12:25:57,2020-07-17 12:30:36,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228340,902C5F7EB021EA07,2020-07-06 17:41:39,2020-07-06 17:49:21,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228341,AB3CA5EDBFF37B0A,2020-07-21 19:02:57,2020-07-21 19:08:02,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228342,CD87E5A1A92EEEA5,2020-07-30 13:22:33,2020-07-30 13:26:04,7th & R St NW / Shaw Library,31245.0,31266.0,11th & M St NW
228343,5CC1323E63436169,2020-07-13 15:04:39,2020-07-13 15:21:04,3rd & H St NE,31616.0,31266.0,11th & M St NW
228344,4399B2F5C1F60345,2020-07-25 19:12:28,2020-07-25 19:21:19,3rd & H St NE,31616.0,31266.0,11th & M St NW
228345,BD2AA5E53A421B93,2020-07-26 20:12:36,2020-07-26 20:25:32,1st & M St NE,31603.0,31266.0,11th & M St NW
228346,C4CC80BCC5AAE9E6,2020-07-31 17:42:00,2020-07-31 17:51:26,1st & M St NE,31603.0,31266.0,11th & M St NW


#### - join with region data

In [17]:
# Import region-wise station information
loc = pd.read_csv('./location_csv/Capital_Bike_Share_Locations.csv')

In [18]:
loc.columns

Index(['X', 'Y', 'OBJECTID', 'LATITUDE', 'LONGITUDE', 'NAME', 'STATION_TYPE',
       'STATION_ID', 'STATION_STATUS', 'LAST_REPORTED', 'NUM_DOCKS_AVAILABLE',
       'NUM_DOCKS_DISABLED', 'NUM_BIKES_AVAILABLE', 'NUM_EBIKES_AVAILABLE',
       'NUM_BIKES_DISABLED', 'IS_INSTALLED', 'IS_RETURNING', 'IS_RENTING',
       'HAS_KIOSK', 'IOS', 'ANDROID', 'ELECTRIC_BIKE_SURCHARGE_WAIVER',
       'EIGHTD_HAS_KEY_DISPENSER', 'CAPACITY', 'RENTAL_METHODS', 'REGION_ID',
       'GIS_LAST_MOD_DTTM', 'GIS_ID', 'REGION_NAME'],
      dtype='object')

In [19]:
# Select neccessary columns
loc_df = loc[['LATITUDE', 'LONGITUDE', 'NAME', 'STATION_TYPE',
       'STATION_ID', 'STATION_STATUS','CAPACITY', 'REGION_ID','REGION_NAME']]

In [20]:
# join full dataset with region info
df_full_region = df_full.join(loc_df.set_index('NAME'), on='Start station')

In [21]:
# convert start and end date to datetime
df_full_region['Start date'] = pd.to_datetime(df_full_region['Start date'])
df_full_region['End date'] = pd.to_datetime(df_full_region['End date'])

In [22]:
# minimize time-resolution to hour
df_full_region['_datetime'] = df_full_region['Start date'].dt.strftime('%Y-%m-%d %H:00:00')

In [23]:
# add date and year
df_full_region['_date'] = df_full_region['Start date'].dt.strftime('%Y-%m-%d')
df_full_region['_year'] = df_full_region['Start date'].dt.strftime('%Y')

In [69]:
# check the number of data points per region
df_full_region.groupby('REGION_NAME').size()

REGION_NAME
Alexandria, VA                     362503
Arlington, VA                     1049941
Fairfax, VA                         54209
Falls Church, VA                    10858
Montgomery County, MD (North)       70440
Montgomery County, MD (South)      294993
Prince George's County              36084
Washington, DC                   26275382
dtype: int64

#### - region-wsie export to csv ( it could've done it a for loop)

In [25]:
df_Alexandria = df_full_region[df_full_region['REGION_NAME'] == 'Alexandria, VA']
df_Alexandria.to_csv('df_Alexandria.csv')

In [27]:
df_Arlington = df_full_region[df_full_region['REGION_NAME'] == 'Arlington, VA']
df_Arlington.to_csv('df_Arlington.csv')

In [29]:
df_Fairfax = df_full_region[df_full_region['REGION_NAME'] == 'Fairfax, VA']
df_Fairfax.to_csv('df_Fairfax.csv')

In [30]:
df_Falls_Church = df_full_region[df_full_region['REGION_NAME'] == 'Falls Church, VA']
df_Falls_Church.to_csv('df_Falls_Church.csv')

In [31]:
df_Montgomery_N = df_full_region[df_full_region['REGION_NAME'] == 'Montgomery County, MD (North)']
df_Montgomery_N.to_csv('df_Montgomery_N.csv')

In [32]:
df_Montgomery_S = df_full_region[df_full_region['REGION_NAME'] == 'Montgomery County, MD (South)']
df_Montgomery_S.to_csv('df_Montgomery_S.csv')

In [33]:
df_Prince_George = df_full_region[df_full_region['REGION_NAME'] == "Prince George's County"]
df_Prince_George.to_csv('df_Prince_George.csv')

In [34]:
df_Washington = df_full_region[df_full_region['REGION_NAME'] == "Washington, DC"]
df_Washington.to_csv('df_Washington.csv')